This notebook applies an inverted Transformer model using PyTorch & Lightning, to get multi-step quantile energy consumption forecasts.

It also performs & demonstrates the necessary data handling & preprocessing steps to get source & target sequences for the Transformer model.

In [1]:
# ===========================
# Robust Project Bootstrap (Colab/Drive-safe)
# ===========================
import os, sys, pickle, warnings

# --- 프로젝트 루트 설정 ---
PROJECT_ROOT = "C:/Users/Daniel/DeepLearningEnergyForecasting"

def ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True)
    return path

def safe_chdir(path: str, fallback: str = "/content") -> None:
    """작업 디렉토리가 사라졌거나 권한 문제일 때도 안전하게 이동."""
    try:
        if not os.path.exists(path):
            warnings.warn(f"[bootstrap] path not found: {path}  → creating it.")
            ensure_dir(path)
        os.chdir(path)
    except Exception as e:
        warnings.warn(f"[bootstrap] chdir({path}) failed: {e}  → chdir({fallback})")
        ensure_dir(fallback)
        os.chdir(fallback)

# 현재 CWD 점검(사라졌을 수 있음)
try:
    _ = os.getcwd()
except FileNotFoundError:
    os.chdir("/content")

# 프로젝트 루트로 이동(복구 포함)
safe_chdir(PROJECT_ROOT)

# --- sys.path 등록(idempotent) ---
def _add_path(p: str):
    if p and p not in sys.path:
        sys.path.insert(0, p)

_add_path(PROJECT_ROOT)
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
_add_path(SRC_PATH)

print("📂 CWD:", os.getcwd())
print("📁 src/deep_learning exists?:", os.path.exists(os.path.join(SRC_PATH, "deep_learning")))

# --- 공통 경로 상수 ---
BASE_DIR    = PROJECT_ROOT
SAFE_ROOT   = ensure_dir(os.path.join(BASE_DIR, "pl_work"))      # Lightning default_root_dir
OPT_LOG_DIR = ensure_dir(os.path.join(BASE_DIR, "opt_logs"))     # Optuna/CSVLogger 등
FINAL_DIR   = ensure_dir(os.path.join(BASE_DIR, "TimesNet_final_run"))

S1_PATH = os.path.join(BASE_DIR, "TimesNet_study_s1.pkl")
S2_PATH = os.path.join(BASE_DIR, "TimesNet_study_s2.pkl")

# --- 유틸 ---
def load_study(path: str):
    """Optuna study pkl 로드. 없으면 None."""
    try:
        with open(path, "rb") as f:
            return pickle.load(f)
    except Exception:
        return None

def save_study(study, path: str):
    """Optuna study pkl 저장(부모 폴더 보장)."""
    ensure_dir(os.path.dirname(path))
    with open(path, "wb") as f:
        pickle.dump(study, f)

# --- 품질 로그 ---
print("✅ BASE_DIR:", BASE_DIR)
print("✅ SAFE_ROOT:", SAFE_ROOT)
print("✅ OPT_LOG_DIR:", OPT_LOG_DIR)
print("✅ FINAL_DIR:", FINAL_DIR)
print("✅ S1_PATH:", S1_PATH)
print("✅ S2_PATH:", S2_PATH)


📂 CWD: C:\Users\Daniel\DeepLearningEnergyForecasting
📁 src/deep_learning exists?: True
✅ BASE_DIR: C:/Users/Daniel/DeepLearningEnergyForecasting
✅ SAFE_ROOT: C:/Users/Daniel/DeepLearningEnergyForecasting\pl_work
✅ OPT_LOG_DIR: C:/Users/Daniel/DeepLearningEnergyForecasting\opt_logs
✅ FINAL_DIR: C:/Users/Daniel/DeepLearningEnergyForecasting\TimesNet_final_run
✅ S1_PATH: C:/Users/Daniel/DeepLearningEnergyForecasting\TimesNet_study_s1.pkl
✅ S2_PATH: C:/Users/Daniel/DeepLearningEnergyForecasting\TimesNet_study_s2.pkl


In [2]:
%pip install lightning
%pip install optuna
%pip install optuna-integration[pytorch_lightning]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [ ]:
# ==== imports (교체본) ====
import math
import warnings  # 필요 없으면 지워도 됨

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import lightning as L
import optuna
import types, importlib, sys
sys.modules['imp'] = types.SimpleNamespace(reload=importlib.reload)

from lightning.pytorch.callbacks import EarlyStopping   # ✅ 추가
from optuna.integration import PyTorchLightningPruningCallback

from src.deep_learning.preprocessing import SequenceScaler, SequenceDataset, SequenceDatasetWithBld
from src.deep_learning.quantile_loss import QuantileLoss
from src.deep_learning.TimesNet import TimesNet
from src.deep_learning.testing import (
    train_val_split, train_val_split_const_batch_size,
    test_sequences_to_dataframe, plot_actual_predicted,
    plot_sequence_preds, calculate_metrics
)
from src.deep_learning.prediction import predictions_to_dataframe

# (선택) 경고 억제
warnings.filterwarnings("ignore", category=UserWarning, module="lightning")


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
random_seed = 1923

In [ ]:
# Set Torch settings
torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')
L.seed_everything(random_seed, workers = True)
warnings.filterwarnings("ignore", ".*does not have many workers.*")

Global seed set to 1923


In [ ]:
# Plot settings
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

In [ ]:
import os

# --- 1. Google Drive 경로에 있는 데이터 파일을 Colab 로컬로 복사 ---

# 원본 파일이 있는 Google Drive 경로
local_train_path = "C:/Users/Daniel/DeepLearningEnergyForecasting/data/analysis/processed/train_advanced_featured.csv"
local_test_path = "C:/Users/Daniel/DeepLearningEnergyForecasting/data/analysis/processed/test_advanced_featured.csv"

# --- 2. 이제부터는 Colab 로컬 경로에서 데이터를 빠르게 불러옵니다 ---

# 수정 코드
train_df = pd.read_csv(local_train_path)
test_df = pd.read_csv(local_test_path)

print("로컬 경로에서 데이터 로딩 완료.")

로컬 경로에서 데이터 로딩 완료.


In [ ]:
train_df["일시"] = pd.to_datetime(train_df["일시"])
test_df["일시"] = pd.to_datetime(test_df["일시"])

In [ ]:
train_df

,ESS저장용량(kWh),PCS용량(kW),month,건물번호,건물유형_IDC(전화국),건물유형_건물기타,건물유형_공공,건물유형_백화점,건물유형_병원,건물유형_상용,...,연면적(m2),일시,전력소비량(kWh),태양광용량(kW),consumption_lag_24h,trend,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos
0,0.0,0,6,1,False,False,False,False,False,False,...,82912.71,2024-06-02 00:00:00,3870.42,0.0,5794.80,24,0.000000,1.000000,-0.781831,0.623490
1,0.0,0,6,1,False,False,False,False,False,False,...,82912.71,2024-06-02 01:00:00,5733.45,0.0,5591.85,25,0.258819,0.965926,-0.781831,0.623490
2,0.0,0,6,1,False,False,False,False,False,False,...,82912.71,2024-06-02 02:00:00,5689.17,0.0,5338.17,26,0.500000,0.866025,-0.781831,0.623490
3,0.0,0,6,1,False,False,False,False,False,False,...,82912.71,2024-06-02 03:00:00,5579.04,0.0,4554.42,27,0.707107,0.707107,-0.781831,0.623490
4,0.0,0,6,1,False,False,False,False,False,False,...,82912.71,2024-06-02 04:00:00,4113.60,0.0,3602.25,28,0.866025,0.500000,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203971,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-24 19:00:00,3276.00,0.0,2206.92,203995,-0.965926,0.258819,-0.974928,-0.222521
203972,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-24 20:00:00,3197.52,0.0,2064.12,203996,-0.866025,0.500000,-0.974928,-0.222521
203973,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-24 21:00:00,3006.60,0.0,1161.12,203997,-0.707107,0.707107,-0.974928,-0.222521
203974,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-24 22:00:00,2649.72,0.0,2076.36,203998,-0.500000,0.866025,-0.974928,-0.222521


In [ ]:
last_24h = train_df.groupby("건물번호").tail(24)[["건물번호", "일시", "전력소비량(kWh)"]]
last_24h["일시"] = last_24h["일시"] + pd.Timedelta(hours=24)
last_24h = last_24h.rename(columns={"전력소비량(kWh)": "consumption_lag_24h"})
test_df = test_df.merge(last_24h, on=["건물번호", "일시"], how="left")
test_df["consumption_lag_24h"] = test_df["consumption_lag_24h"].fillna(train_df["전력소비량(kWh)"].mean())
test_df = test_df.sort_values(["건물번호", "일시"]).reset_index(drop=True)
test_df["trend"] = range(len(test_df))

# 검증
print("df_test shape:", test_df.shape)
print("df_test columns:", test_df.columns)
print("df_test head:", test_df[["건물번호", "일시", "consumption_lag_24h", "trend"]].head())
print("df_test tail:", test_df[["건물번호", "일시", "consumption_lag_24h", "trend"]].tail())
print("Missing consumption_lag_24h:", test_df["consumption_lag_24h"].isna().sum())
test_df

df_test shape: (16800, 26)
df_test columns: Index(['ESS저장용량(kWh)', 'PCS용량(kW)', 'month', '건물번호', '건물유형_IDC(전화국)',
       '건물유형_건물기타', '건물유형_공공', '건물유형_백화점', '건물유형_병원', '건물유형_상용', '건물유형_아파트',
       '건물유형_연구소', '건물유형_학교', '건물유형_호텔', '기온(°C)', '냉방면적(m2)', '연면적(m2)', '일시',
       '전력소비량(kWh)', '태양광용량(kW)', 'hour_sin', 'hour_cos', 'day_of_week_sin',
       'day_of_week_cos', 'consumption_lag_24h', 'trend'],
      dtype='object')
df_test head:    건물번호                  일시  consumption_lag_24h  trend
0     1 2024-08-25 00:00:00              4897.59      0
1     1 2024-08-25 01:00:00              4664.22      1
2     1 2024-08-25 02:00:00              3716.73      2
3     1 2024-08-25 03:00:00              3073.47      3
4     1 2024-08-25 04:00:00              3399.63      4
df_test tail:        건물번호                  일시  consumption_lag_24h  trend
16795   100 2024-08-31 19:00:00          3329.432482  16795
16796   100 2024-08-31 20:00:00          3329.432482  16796
16797   100 2024-08-31 21:0

,ESS저장용량(kWh),PCS용량(kW),month,건물번호,건물유형_IDC(전화국),건물유형_건물기타,건물유형_공공,건물유형_백화점,건물유형_병원,건물유형_상용,...,연면적(m2),일시,전력소비량(kWh),태양광용량(kW),hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,consumption_lag_24h,trend
0,0.0,0,8,1,False,False,False,False,False,False,...,82912.71,2024-08-25 00:00:00,0,0.0,0.000000,1.000000,-0.781831,0.623490,4897.590000,0
1,0.0,0,8,1,False,False,False,False,False,False,...,82912.71,2024-08-25 01:00:00,0,0.0,0.258819,0.965926,-0.781831,0.623490,4664.220000,1
2,0.0,0,8,1,False,False,False,False,False,False,...,82912.71,2024-08-25 02:00:00,0,0.0,0.500000,0.866025,-0.781831,0.623490,3716.730000,2
3,0.0,0,8,1,False,False,False,False,False,False,...,82912.71,2024-08-25 03:00:00,0,0.0,0.707107,0.707107,-0.781831,0.623490,3073.470000,3
4,0.0,0,8,1,False,False,False,False,False,False,...,82912.71,2024-08-25 04:00:00,0,0.0,0.866025,0.500000,-0.781831,0.623490,3399.630000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-31 19:00:00,0,0.0,-0.965926,0.258819,-0.974928,-0.222521,3329.432482,16795
16796,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-31 20:00:00,0,0.0,-0.866025,0.500000,-0.974928,-0.222521,3329.432482,16796
16797,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-31 21:00:00,0,0.0,-0.707107,0.707107,-0.974928,-0.222521,3329.432482,16797
16798,0.0,0,8,100,False,False,False,False,False,False,...,162070.24,2024-08-31 22:00:00,0,0.0,-0.500000,0.866025,-0.974928,-0.222521,3329.432482,16798


## Data prep: Getting input & output sequences

We will create a "shifted dataset" where each row at time T contains the following columns:
- Target value at T (consumption lead T+1),
- Past value at T (consumption lag T-2),
- Time covariates at T.

Every source sequence will be the last 72 hours before 16:00, and every target sequence will be the next 32 hours after 16:00.

In [ ]:
# ===================================================================
# 1. 최종 파라미터 및 피처 리스트 정의
# ===================================================================
SOURCE_LENGTH = 168
TARGET_LENGTH = 168
TARGET_COL = '전력소비량(kWh)'
BUILDING_COL = '건물번호'  # ✅ 건물 ID 명시

# 시간에 따라 변하는 변수들 (TARGET_COL 제외)
FEATURE_COLS = [
    '기온(°C)',
    'consumption_lag_24h', 'trend',
    'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos'
]

# 건물별로 고정된 변수들
STATIC_COLS = [col for col in train_df.columns if "건물유형_" in col or "면적" in col or "용량" in col]

# (중요) 스케일링/모델 입력 컬럼 최종 순서: [건물번호] + 동적 + 정적 + [타깃]
FINAL_COLS_ORDERED = [BUILDING_COL] + FEATURE_COLS + STATIC_COLS + [TARGET_COL]

print("✅ 파라미터 및 피처 리스트 정의 완료.")
print(f"총 입력 채널 수(건물번호 포함): {len(FINAL_COLS_ORDERED)}")

# ===================================================================
# 2. 다중 시계열 Transformer용 시퀀스 생성 함수 (건물번호 보존 버전)
# ===================================================================
def create_transformer_sequences_for_prediction(
    train_df,
    test_df,
    feature_cols,
    static_cols,
    target_col,
    source_length,
    target_length,
    mode: str = "inference",   # "inference" or "train"
    stride: int = 1            # train 모드에서 롤링 간격
):
    import numpy as np

    assert mode in ("inference", "train"), "mode must be 'inference' or 'train'"
    source_sequences, target_sequences = [], []

    train_df = train_df.sort_values([BUILDING_COL, '일시'])
    if mode == "inference":
        test_df = test_df.sort_values([BUILDING_COL, '일시'])

    final_cols_ordered = [BUILDING_COL] + feature_cols + static_cols + [target_col]

    for building_id in train_df[BUILDING_COL].unique():
        g_train = train_df[train_df[BUILDING_COL] == building_id]

        # 정적값(건물 단위로 동일하다는 가정)
        static_vals = g_train.iloc[0][static_cols]

        if mode == "inference":
            g_test = test_df[test_df[BUILDING_COL] == building_id]

            # --- Source (train tail) ---
            if len(g_train) < source_length:
                continue
            source_seq = g_train.tail(source_length).copy()

            # --- Target (test head) ---
            if len(g_test) < target_length:
                continue
            target_seq = g_test.head(target_length).copy()

            # ---- 타깃 초기값: 선형 외삽 ----
            y_hist = source_seq[target_col].to_numpy(dtype=np.float32)  # (L,)
            L, H = source_length, target_length
            x = np.arange(L, dtype=np.float32)
            A = np.vstack([x, np.ones_like(x)]).T
            m, c = np.linalg.lstsq(A, y_hist, rcond=None)[0]
            fut_idx = np.arange(L, L + H, dtype=np.float32)
            y_init = (m * fut_idx + c).astype(np.float32)
            if not np.isfinite(y_init).all():
                y_init = np.full(H, y_hist[-1], dtype=np.float32)
            target_seq[target_col] = y_init  # 외삽값으로 채움 (y_true 없음)

            # --- 정적 피처 브로드캐스트 ---
            for col in static_cols:
                source_seq[col] = static_vals[col]
                target_seq[col] = static_vals[col]

            # ✅ 건물번호 명시 보존 (혹시 누락 방지)
            source_seq[BUILDING_COL] = building_id
            target_seq[BUILDING_COL] = building_id

            # 시간 인덱스만 설정 (건물번호는 컬럼으로 남김)
            source_sequences.append(source_seq.set_index('일시')[final_cols_ordered])
            target_sequences.append(target_seq.set_index('일시')[final_cols_ordered])

        else:  # mode == "train"
            total_len = len(g_train)
            if total_len < source_length + target_length:
                continue

            g_train = g_train.reset_index(drop=True)
            for start in range(0, total_len - (source_length + target_length) + 1, stride):
                s0 = start
                s1 = start + source_length
                t1 = s1 + target_length

                source_seq = g_train.iloc[s0:s1].copy()
                target_seq = g_train.iloc[s1:t1].copy()

                # 정적 피처 브로드캐스트
                for col in static_cols:
                    source_seq[col] = static_vals[col]
                    target_seq[col] = static_vals[col]

                # ✅ 건물번호 명시 보존
                source_seq[BUILDING_COL] = building_id
                target_seq[BUILDING_COL] = building_id

                source_sequences.append(source_seq.set_index('일시')[final_cols_ordered])
                target_sequences.append(target_seq.set_index('일시')[final_cols_ordered])

    return source_sequences, target_sequences

# ===================================================================
# 3. 함수 실행 및 결과 확인
# ===================================================================
train_src, train_tgt = create_transformer_sequences_for_prediction(
    train_df, test_df,
    FEATURE_COLS, STATIC_COLS, TARGET_COL,
    SOURCE_LENGTH, TARGET_LENGTH,
    mode="train",
    stride=24
)
infer_src, infer_tgt = create_transformer_sequences_for_prediction(
    train_df, test_df,
    FEATURE_COLS, STATIC_COLS, TARGET_COL,
    SOURCE_LENGTH, TARGET_LENGTH,
    mode="inference"
)

print(f"\n✅ Transformer용 시퀀스 생성 완료!")
print(f"Source 시퀀스 개수: {len(train_src)}")
print(f"Target 시퀀스 개수: {len(train_tgt)}")
print("\n--- 첫 번째 건물의 Source Sequence (마지막 5행) ---")
display(train_src[0].tail())
print("\n--- 첫 번째 건물의 Target Sequence (처음 5행) ---")
display(train_tgt[0].head())


✅ 파라미터 및 피처 리스트 정의 완료.
총 입력 채널 수(건물번호 포함): 24

✅ Transformer용 시퀀스 생성 완료!
Source 시퀀스 개수: 7199
Target 시퀀스 개수: 7199

--- 첫 번째 건물의 Source Sequence (마지막 5행) ---


,건물번호,기온(°C),consumption_lag_24h,trend,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,ESS저장용량(kWh),PCS용량(kW),...,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_학교,건물유형_호텔,냉방면적(m2),연면적(m2),태양광용량(kW),전력소비량(kWh)
일시,,,,,,,,,,,,,,,,,,,,,
2024-06-08 19:00:00,1,23.2,6125.85,187,-0.965926,0.258819,-0.974928,-0.222521,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5333.49
2024-06-08 20:00:00,1,22.0,5478.39,188,-0.866025,0.500000,-0.974928,-0.222521,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5411.25
2024-06-08 21:00:00,1,21.2,4299.99,189,-0.707107,0.707107,-0.974928,-0.222521,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,4292.37
2024-06-08 22:00:00,1,20.6,3914.49,190,-0.500000,0.866025,-0.974928,-0.222521,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,4599.39
2024-06-08 23:00:00,1,20.2,5652.33,191,-0.258819,0.965926,-0.974928,-0.222521,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5150.10



--- 첫 번째 건물의 Target Sequence (처음 5행) ---


,건물번호,기온(°C),consumption_lag_24h,trend,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,ESS저장용량(kWh),PCS용량(kW),...,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_학교,건물유형_호텔,냉방면적(m2),연면적(m2),태양광용량(kW),전력소비량(kWh)
일시,,,,,,,,,,,,,,,,,,,,,
2024-06-09 00:00:00,1,19.5,4960.32,192,0.000000,1.000000,-0.781831,0.62349,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5350.35
2024-06-09 01:00:00,1,19.0,4391.73,193,0.258819,0.965926,-0.781831,0.62349,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5523.81
2024-06-09 02:00:00,1,18.6,5055.51,194,0.500000,0.866025,-0.781831,0.62349,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5441.73
2024-06-09 03:00:00,1,18.1,5389.44,195,0.707107,0.707107,-0.781831,0.62349,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,5365.44
2024-06-09 04:00:00,1,17.7,5146.26,196,0.866025,0.500000,-0.781831,0.62349,0.0,0,...,False,False,False,False,False,True,77586.0,82912.71,0.0,4025.13


Each source sequence will contain the following:
- Consumption lag 2 at [T - source_length, T] (past targets),
- Time covariates at [T - source_length, T] (past covariates).

Each output sequence will contain the following:
- Consumption at [T+1, T + output_length] (future targets),
- Time covariates at [T+1, T + output_length] (future known covariates)

In the source sequence, we pair every past target value at T with the past covariates at T.
\
In the output sequence, we pair every future target value at T+1 with the future covariates at T+1.

This is more straightforward than the LSTM sequencing in notebook 4.0, because we will make our predictions in one go, and don't need to combine predictions at T+1 with future covariates at T+2 to get the input sequence for T+1.

We will simply initialize the future targets from T+1 onwards as a linear extrapolation of the past targets from T - source_length to T. We'll do this in training as well as validation & prediction.
\
The idea is that the model should learn to "enrich" a simple linear extrapolation given additional future known covariates.

In [ ]:
len(train_src)

7199

In [ ]:
len(train_tgt)

7199

## Preprocessing: Custom scaler, Torch datasets & dataloaders

We'll keep batch sizes constant to match the evaluation scheme with the stateful LSTM in notebook 4.0, though there should be no need with this model.
\
We'll divide the lengths of each data fold with the batch size, drop the remainder from the start of the training folds & the end of the val. & test folds.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch

# ===================================================================
# 0) 컬럼 구성: "스케일러용"과 "모델입력용"을 분리
# ===================================================================
BUILDING_COL = "건물번호"
TARGET_COL   = "전력소비량(kWh)"

# 스케일러/시퀀스 DF에는 건물번호를 포함해둠(그룹별 통계 계산용)
FINAL_COLS_WITH_BUILDING = [BUILDING_COL] + FEATURE_COLS + STATIC_COLS + [TARGET_COL]

# 모델 입력 텐서는 '건물번호' 채널을 제거하고 사용
FINAL_COLS_FOR_MODEL = FEATURE_COLS + STATIC_COLS + [TARGET_COL]   # (타깃은 항상 마지막)

# ===================================================================
# 1. 데이터 분할
# ===================================================================
train_source, train_target, val_source, val_target = train_val_split_const_batch_size(
    train_src,
    train_tgt,
    train_fraction=0.9,
)
print("데이터 분할 완료")
print(f"학습용 데이터: {len(train_source)}개, 검증용 데이터: {len(val_source)}개")

# 0..NUM_BUILDINGS-1 매핑
uniq_ids = sorted(train_df[BUILDING_COL].unique().tolist())
bld2idx = {bid: i for i, bid in enumerate(uniq_ids)}

# 분할 직후, bld2idx 만들고 난 다음에 바로 추가
group_ids_val = [df.iloc[0][BUILDING_COL] for df in val_target]
print("✅ group_ids_val 준비 완료: ", len(group_ids_val))

# 각 시퀀스의 건물 id → index
train_bld_ids_idx = np.array([bld2idx[int(df[BUILDING_COL].iloc[0])] for df in train_source], dtype=np.int64)
val_bld_ids_idx   = np.array([bld2idx[int(df[BUILDING_COL].iloc[0])] for df in val_source],   dtype=np.int64)
infer_bld_ids_idx = np.array([bld2idx[int(df[BUILDING_COL].iloc[0])] for df in infer_src],    dtype=np.int64)

import json
with open("C:/Users/Daniel/DeepLearningEnergyForecasting/bld2idx.json", "w") as f:
    json.dump({str(k): int(v) for k, v in bld2idx.items()}, f)

# ===================================================================
# 2. 스케일러 정의 및 학습
#    - 핵심: 타깃을 '건물별' MinMax(0–1)로 정규화할 준비
#    - SequenceScaler는 내부에서 target을 마지막 컬럼으로 강제 정렬한다고 가정
# ===================================================================

# 👉 STATIC_COLS는 이미 DF에 포함됩니다. 여기의 feature 리스트는 "스케일 방식"만 지정하는 목적입니다.
#    (중복 지정해도 내부에서 중복 제거/우선순위 처리하도록 설계되어 있음)
standard_features = ['기온(°C)', 'consumption_lag_24h', 'trend']  # (권장) 순수 동적만 표준화
# 면적/용량 같은 정적은 대체로 MinMax 또는 그대로 두어도 됨. 아래는 예시:
minmax_features   = ['연면적(m2)', '냉방면적(m2)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '태양광용량(kW)']

# 사인/코사인/요일 임베딩은 그대로 통과
passthrough_features = ['hour_sin','hour_cos','day_of_week_sin','day_of_week_cos']

# 건물별 타깃 정규화 옵션 추가 (클래스 내부 구현 예정)
scaler = SequenceScaler(
    target_col=TARGET_COL,
    standard_scale_cols=standard_features,
    minmax_scale_cols=minmax_features,
    passthrough_cols=passthrough_features,
    per_building=True,              # 건물별로 타깃 정규화 통계를 가짐
    groupby_key=BUILDING_COL,       # 그룹핑 키
    target_minmax=(0.0, 1.0),       # 타깃 스케일 범위
    use_group_for=['target']        # (선택) 타깃만 그룹별 정규화
)
print("\nFitting the scaler on the training data...")

# (id, df) 튜플 형태로 전달 (df는 '건물번호' 컬럼을 포함)
train_source_with_ids = list(enumerate(train_source))
train_target_with_ids = list(enumerate(train_target))

scaler.fit(train_source_with_ids, train_target_with_ids)

print("Transforming all datasets...")
val_source_with_ids = list(enumerate(val_source))
val_target_with_ids = list(enumerate(val_target))

train_source_scaled = scaler.transform(train_source_with_ids)
train_target_scaled = scaler.transform(train_target_with_ids)
val_source_scaled   = scaler.transform(val_source_with_ids)
val_target_scaled   = scaler.transform(val_target_with_ids)

print("✅ 피처별 스케일링 적용 (타깃: 건물별 MinMax 준비 완료)")

# ===================================================================
# 3. 데이터로더 생성
#    - 모델 입력에서 '건물번호' 채널을 제거 (스케일러는 사용했지만 모델에는 아직 투입하지 않음)
# ===================================================================

# 스케일러는 원 DF 컬럼 순서를 유지하되 타깃을 마지막으로 둔다고 가정
# 우리는 첫 번째 열(건물번호)을 드롭하고 모델 입력용 배열을 만든다.
def drop_building_col(arr: np.ndarray) -> np.ndarray:
    # arr shape: (T, D_with_building); building col이 0번, target은 마지막
    return arr[:, 1:]  # 건물번호 제거

train_source_arrays_full = [seq_arr for _, seq_arr in train_source_scaled]
train_target_arrays_full = [seq_arr for _, seq_arr in train_target_scaled]
val_source_arrays_full   = [seq_arr for _, seq_arr in val_source_scaled]
val_target_arrays_full   = [seq_arr for _, seq_arr in val_target_scaled]

# 모델 입력용으로 building col 제거
train_source_arrays = [drop_building_col(a) for a in train_source_arrays_full]
train_target_arrays = [drop_building_col(a) for a in train_target_arrays_full]
val_source_arrays   = [drop_building_col(a) for a in val_source_arrays_full]
val_target_arrays   = [drop_building_col(a) for a in val_target_arrays_full]

# 검증: 타깃이 마지막인지, 모델용 최종 컬럼 수가 맞는지
target_col_index = FINAL_COLS_FOR_MODEL.index(TARGET_COL)
assert train_target_arrays[0].shape[1] - 1 == target_col_index
assert FINAL_COLS_FOR_MODEL[-1] == TARGET_COL
assert train_target_arrays[0].shape[1] == len(FINAL_COLS_FOR_MODEL)

print("Creating PyTorch Datasets...")
train_data = SequenceDatasetWithBld(np.array(train_source_arrays),
                                    np.array(train_target_arrays),
                                    train_bld_ids_idx)
val_data   = SequenceDatasetWithBld(np.array(val_source_arrays),
                                    np.array(val_target_arrays),
                                    val_bld_ids_idx)
print("✅ Datasets created successfully.")

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_data,   batch_size=batch_size, shuffle=False)

print("\n✅ Transformer용 DataLoader가 성공적으로 생성되었습니다.")
first_src, first_tgt, first_bld = next(iter(train_loader))
print(first_src.shape, first_tgt.shape, first_bld.shape)

# 0) 예측 전용(Validation) DataLoader 준비
#    - predict_step은 target에 '타깃 제외(covariates-only)' 배열을 기대 → 마지막 컬럼(타깃) 제거
val_target_cov_arrays = [arr[:, :-1] for arr in val_target_arrays]
val_pred_data = SequenceDatasetWithBld(
    np.array(val_source_arrays),
    np.array(val_target_cov_arrays),
    bld_ids=val_bld_ids_idx
)
val_pred_loader = torch.utils.data.DataLoader(val_pred_data, batch_size=batch_size, shuffle=False)
print("✅ val_pred_loader 준비 완료 (predict 전용)")

데이터 분할 완료
학습용 데이터: 6464개, 검증용 데이터: 704개
✅ group_ids_val 준비 완료:  704

Fitting the scaler on the training data...
Transforming all datasets...
✅ 피처별 스케일링 적용 (타깃: 건물별 MinMax 준비 완료)
Creating PyTorch Datasets...
✅ Datasets created successfully.

✅ Transformer용 DataLoader가 성공적으로 생성되었습니다.
torch.Size([64, 168, 23]) torch.Size([64, 168, 23]) torch.Size([64])
✅ val_pred_loader 준비 완료 (predict 전용)


We have to scale the past consumption values in the source sequences, and covariates in both source & target sequences.
\
We also need to scale the real target consumption values because they are used in the loss calculation, even if not passed as input to the model.
\
We also need the ability to backtransform the network's final predictions accordingly. Again, we use `SequenceScaler`.

We can use one Torch dataset class both for training & validation (known future targets part of passed target sequences), and for prediction (no known future targets in the target sequences).
\
In the former case, both the source & target sequence need to have D feature dimensions (including targets). In the latter case, the target sequence needs to have D-1 feature dimensions, without targets.

## Model definition

The `LITransformer` model is implemented in `src.deep_learning.transformer`.

In [ ]:
# 파일 최상단 (한 번만)
import math, numpy as np, torch, optuna, pickle  # ← pickle 추가
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

PUBLIC_H = 72
EPS_PUBLIC = 1.0
NONNEG_CLIP = True
num_buildings = len(uniq_ids)

def _smape_np(y_true, y_pred, eps=EPS_PUBLIC):
    y_true = np.asarray(y_true, np.float32)
    y_pred = np.asarray(y_pred, np.float32)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    if not np.any(m): return 1e9
    y_true = y_true[m]; y_pred = y_pred[m]
    return float(np.mean(2.0*np.abs(y_pred - y_true) /
                         (np.abs(y_true) + np.abs(y_pred) + eps)))

def _public_tail(arr, Hpub):
    return arr[:, -Hpub:] if arr.shape[1] >= Hpub else arr


In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import CSVLogger

def stage1_objective(trial, source_dims, target_dims):
    """Stage-1: TimesNet 용량/구조 + 일부 학습 하이퍼 탐색 (proxy: val_loss)"""

    # ===== TimesNet 구조 하이퍼(확장) =====
    d_model_exp  = trial.suggest_int("d_model_exp", 7, 9)   # {128,256,512}
    d_model      = 2 ** d_model_exp
    n_encoders   = trial.suggest_int("n_encoders", 1, 3)
    n_decoders   = 1  # Stage-1 프록시는 1로 고정(속도↑, Stage-2/Final에서 decoder 늘려도 됨)

    # TimesNet 고유
    top_k        = trial.suggest_categorical("top_k", [2, 3, 4])
    num_kernels  = trial.suggest_categorical("num_kernels", [4, 6, 8])
    d_ff_mult    = trial.suggest_categorical("d_ff_mult", [2, 3, 4])
    d_ff_tn      = d_model * d_ff_mult

    # ===== 학습 관련 =====
    learning_rate = trial.suggest_float("learning_rate", 3e-4, 2e-3, log=True)
    dropout_rate  = trial.suggest_float("dropout_rate", 0.0, 0.4)
    lr_decay      = trial.suggest_float("lr_decay", 0.985, 1.0)
    d_bld         = trial.suggest_categorical("d_bld", [8, 16, 32])
    emb_dropout   = trial.suggest_float("emb_dropout", 0.0, 0.2)
    grad_clip     = trial.suggest_float("gradient_clip_val", 0.5, 2.0)
    accum         = trial.suggest_categorical("accumulate_grad_batches", [1, 2, 4])

    # S2에서 동작 튜닝할 값은 Stage-1 고정(프록시 단순화)
    beta_week = 0.7
    alpha     = 0.8
    use_adaptive_beta = trial.suggest_categorical("use_adaptive_beta", [True, False])

    h = {
        "source_length": SOURCE_LENGTH, "target_length": TARGET_LENGTH,
        "source_dims": source_dims, "target_dims": target_dims,
        "horizon_start": 0, "quantiles": [0.025, 0.5, 0.975],

        "d_model": d_model, "n_encoders": n_encoders, "n_decoders": n_decoders,
        "learning_rate": learning_rate, "dropout_rate": dropout_rate, "lr_decay": lr_decay,

        "num_buildings": len(uniq_ids),
        "d_bld": d_bld, "emb_dropout": emb_dropout,

        # TimesNet 전용
        "top_k": top_k, "num_kernels": num_kernels, "d_ff_tn": d_ff_tn,

        # Stage-1 고정(동작)
        "beta_week": beta_week, "alpha": alpha,
        "use_adaptive_beta": use_adaptive_beta,
    }

    model = TimesNet(h)

    # ===== 콜백/로거 =====
    pruner_cb = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    # proxy 학습 조금 더 길게 → patience=3
    es_cb = EarlyStopping(monitor="val_loss", mode="min", patience=3, check_on_train_epoch_end=False)
    logger = CSVLogger(OPT_LOG_DIR, name=f"s1_tn_trial{trial.number}")

    # ===== Trainer: 프록시 세팅(늘림) =====
    trainer = Trainer(
        default_root_dir=SAFE_ROOT,       # 안전 루트
        max_epochs=30,
        accelerator="cpu", devices=1,
        precision=32,
        logger=logger,
        enable_progress_bar=False,
        enable_checkpointing=False,
        gradient_clip_val=grad_clip,
        accumulate_grad_batches=accum,
        check_val_every_n_epoch=1,
        # 프록시 샘플 비율 확장(학습 60%, 검증 100%)
        limit_train_batches=0.6,
        limit_val_batches=1.0,
        callbacks=[pruner_cb, es_cb],
        deterministic=False, benchmark=True,
        num_sanity_val_steps=0,
    )

    # ===== 학습 =====
    trainer.fit(model, train_loader, val_loader)

    # ===== 베스트 val_loss 안전 취득 =====
    val_loss_obj = getattr(es_cb, "best_score", None)
    if val_loss_obj is None:
        val_loss_obj = trainer.callback_metrics.get("val_loss", None)

    if val_loss_obj is None:
        best_val = float("inf")
    else:
        best_val = float(val_loss_obj.detach().cpu().item() if hasattr(val_loss_obj, "detach") else val_loss_obj)

    # 로깅
    trial.set_user_attr("n_epochs", trainer.current_epoch + 1)
    trial.set_user_attr("val_loss", best_val)
    # 모델 용량 로그(사후 분석용)
    trial.set_user_attr("param_count_M", sum(p.numel() for p in model.parameters()) / 1e6)

    return best_val

#임시 코드
import os
if os.path.exists(S1_PATH):
    os.remove(S1_PATH)


In [ ]:
def stage2_objective(trial, source_dims, target_dims, s1_params):
    """
    Stage-2: 동작 하이퍼(베이스라인/로스 혼합)만 탐색 — objective: public SMAPE
    - 구조/용량 하이퍼는 Stage-1 결과를 고정 사용
    """
    # --- Stage-1에서 고정할 구조/용량 하이퍼 가져오기 ---
    # d_model
    d_model = s1_params.get("d_model", 2 ** s1_params.get("d_model_exp", 7))  # fallback 128
    # 깊이
    n_encoders = s1_params.get("n_encoders", 2)
    n_decoders = s1_params.get("n_decoders", max(1, n_encoders))
    # TimesNet 전용
    top_k       = s1_params.get("top_k", 3)
    num_kernels = s1_params.get("num_kernels", 6)
    d_ff_tn = s1_params.get("d_ff_tn", int(d_model * 3))
    # 학습 관련
    learning_rate = s1_params.get("learning_rate", 1e-3)
    dropout_rate  = s1_params.get("dropout_rate", 0.2)
    lr_decay      = s1_params.get("lr_decay", 1.0)
    grad_clip     = s1_params.get("gradient_clip_val", 1.0)
    accum         = s1_params.get("accumulate_grad_batches", 1)
    # 임베딩
    d_bld       = s1_params.get("d_bld", 16)
    emb_dropout = s1_params.get("emb_dropout", 0.0)

    # --- Stage-2 탐색: 동작 하이퍼 ---
    beta_week = trial.suggest_float("beta_week", 0.55, 0.85)
    alpha     = trial.suggest_float("alpha",     0.70, 0.85)
    # 적응형 β on/off 어블레이션 (원하면 True 고정도 가능)
    use_adaptive_beta = trial.suggest_categorical("use_adaptive_beta", [True, False])

    # === Hyperparams dict ===
    h = {
        "source_length": SOURCE_LENGTH, "target_length": TARGET_LENGTH,
        "source_dims": source_dims,     "target_dims": target_dims,
        "horizon_start": 0,             "quantiles": [0.025, 0.5, 0.975],

        # 구조/용량 (Stage-1 고정)
        "d_model": d_model,
        "n_encoders": n_encoders,
        "n_decoders": n_decoders,

        # 학습 관련
        "learning_rate": learning_rate,
        "dropout_rate":  dropout_rate,
        "lr_decay":      lr_decay,

        # 임베딩
        "num_buildings": num_buildings,
        "d_bld": d_bld,
        "emb_dropout": emb_dropout,

        # TimesNet 전용
        "top_k": top_k,
        "num_kernels": num_kernels,
        "d_ff_tn": d_ff_tn,

        # 동작 하이퍼(탐색)
        "beta_week": beta_week,
        "alpha": alpha,
        "use_adaptive_beta": use_adaptive_beta,
    }

    # === Model ===
    model = TimesNet(h)

    # 콜백/프루너: 짧은 에폭에 맞게 완화
    es_cb = EarlyStopping(
        monitor="val_loss", mode="min",
        patience=2, check_on_train_epoch_end=False
    )
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=10,      # 초반 8개는 살려서 분포 파악
        n_warmup_steps=1,        # 첫 에폭은 버퍼
        interval_steps=1         # 매 에폭마다 프루닝 판정
    )
    pruner_cb = PyTorchLightningPruningCallback(trial, monitor="val_loss")

    trainer = Trainer(
        max_epochs=5,
        accelerator="cpu", devices=1,
        precision=32,
        logger=False, enable_checkpointing=False,
        enable_progress_bar=False,
        gradient_clip_val=s1_params.get("gradient_clip_val", 1.0),
        accumulate_grad_batches=s1_params.get("accumulate_grad_batches", 1),
        check_val_every_n_epoch=1,
        # 속도↑ + 검증 신호 안정성(80%)
        limit_train_batches=0.6,
        limit_val_batches=0.8,
        callbacks=[es_cb, pruner_cb],
        deterministic=False, benchmark=True,
        num_sanity_val_steps=0,
    )

    trainer.fit(model, train_loader, val_loader)

    try:
        # === 예측 산출 ===
        model_eval = model.eval()
        with torch.inference_mode():
            raw = trainer.predict(model_eval, dataloaders=val_pred_loader)
        preds_raw_list, fut_ex_list = zip(*raw)
        residual_q = torch.cat(preds_raw_list, dim=0).cpu().numpy().astype(np.float32)  # (N,H,Q)
        fut_ex     = torch.cat(fut_ex_list,   dim=0).cpu().numpy().astype(np.float32)   # (N,H)

        # === 타깃 스케일 공간 준비 ===
        mid = getattr(model_eval, "median_idx", 1)
        y_scaled_full = np.stack([arr[:, -1] for arr in val_target_arrays], axis=0).astype(np.float32)

        # 길이 동기화
        n = min(residual_q.shape[0], len(group_ids_val), y_scaled_full.shape[0], fut_ex.shape[0])
        y_scaled_used  = y_scaled_full[:n]
        fut_ex_used    = fut_ex[:n]
        group_ids_used = list(group_ids_val[:n])

        # ===== (A) 스케일 공간 바이어스 추정 (중앙 분위수 기준) =====
        got  = residual_q[:n, :, mid]                 # (N,H)
        need = y_scaled_used - fut_ex_used            # (N,H)
        residual_bias = float((got - need).mean())    # scalar

        # ===== (B) 보정 전/후 두 버전 모두 평가 =====
        yhat_scaled_full_raw  = got + fut_ex_used                 # 보정 전
        yhat_scaled_full_bias = (got - residual_bias) + fut_ex_used  # 보정 후

        Hpub = min(int(PUBLIC_H), y_scaled_used.shape[1])

        def _eval_smape(yhat_scaled_full):
            yhat_scaled = yhat_scaled_full[:, -Hpub:]
            y_scaled    = y_scaled_used[:, -Hpub:]
            yhat_real = scaler.backtransform_preds(yhat_scaled, group_ids=group_ids_used)
            y_real    = scaler.backtransform_preds(y_scaled,    group_ids=group_ids_used)
            yhat_real = np.clip(yhat_real, 0.0, None); y_real = np.clip(y_real, 0.0, None)
            sm = _smape_np(y_real, yhat_real, eps=EPS_PUBLIC)
            bs = float(np.nanmean(yhat_real - y_real))
            return sm, bs

        sm0, bs0 = _eval_smape(yhat_scaled_full_raw)    # 보정 전
        sm1, bs1 = _eval_smape(yhat_scaled_full_bias)   # 보정 후

        # ===== (C) 더 좋은 쪽 채택 =====
        if sm1 <= sm0:
            smape_pub, bias_pub = sm1, bs1
            used_bias = residual_bias
        else:
            smape_pub, bias_pub = sm0, bs0
            used_bias = 0.0

        # (선택) 큰 편향 페널티
        if abs(bias_pub) > 0.05 * (np.nanmean(y_scaled_used[:, -Hpub:]) + 1e-6):
            smape_pub *= 1.01

        # Optuna 기록
        val_loss_tensor = trainer.callback_metrics.get("val_loss", torch.tensor(float("inf")))
        trial.set_user_attr("val_loss", float(val_loss_tensor.detach().cpu().item() if hasattr(val_loss_tensor, "detach") else val_loss_tensor))
        trial.set_user_attr("smape_public", smape_pub)
        trial.set_user_attr("bias_public", bias_pub)
        trial.set_user_attr("bias_used_in_eval", used_bias)
        trial.set_user_attr("smape_no_bias", sm0)
        trial.set_user_attr("smape_with_bias", sm1)
        trial.set_user_attr("public_H_used", Hpub)
        trial.set_user_attr("beta_week", beta_week)
        trial.set_user_attr("alpha", alpha)
        trial.set_user_attr("use_adaptive_beta", use_adaptive_beta)

        return smape_pub

    except Exception as e:
        trial.set_user_attr("public_eval_error", str(e))
        return float("inf")

#임시 코드
import os
if os.path.exists(S2_PATH):
    os.remove(S2_PATH)

## Hyperparameter tuning with Optuna

In [ ]:
# 실측 입력 차원
actual_source_dims = train_source_arrays[0].shape[1]
actual_target_dims = train_target_arrays[0].shape[1]
print(f"Source Dims: {actual_source_dims}, Target Dims: {actual_target_dims}")

RUN_STAGE1 = True  # 다음에 다시 돌릴 때 여기만 토글

study_s1 = load_study(S1_PATH)

if (study_s1 is None) and RUN_STAGE1:
    # Hyperband: 길고 넓은 탐색에 유리 (R→reduction_factor로 강도 조절)
    pruner = optuna.pruners.HyperbandPruner(min_resource=1, reduction_factor=3)
    study_s1 = optuna.create_study(
        sampler=optuna.samplers.TPESampler(seed=random_seed, multivariate=True, group=True),
        pruner=pruner,
        study_name="TimesNet_stage1_capacity",
        direction="minimize",
    )

    N_TRIALS = 150   # ← 충분히 늘림(예: 150~300)
    study_s1.optimize(
        lambda tr: stage1_objective(tr, actual_source_dims, actual_target_dims),
        n_trials=N_TRIALS,
        show_progress_bar=True,
        gc_after_trial=True,
        n_jobs=1,  # Colab 단일 GPU 기준
    )

    save_study(study_s1, S1_PATH)

elif study_s1 is None and not RUN_STAGE1:
    raise RuntimeError(f"RUN_STAGE1=False 이지만 저장된 Stage-1 스터디가 없습니다: {S1_PATH}")

# 요약 출력
best_s1 = study_s1.best_trial
print("S1 best val_loss:", best_s1.value)
print("S1 best params:", best_s1.params)
print("S1 attrs:", getattr(best_s1, "user_attrs", {}))
s1p = best_s1.params

c:\Users\Daniel\anaconda3\envs\aiot\lib\site-packages\optuna\_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
c:\Users\Daniel\anaconda3\envs\aiot\lib\site-packages\optuna\_experimental.py:32: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-08-21 00:48:26,793] A new study created in memory with name: TimesNet_stage1_capacity


Source Dims: 23, Target Dims: 23


  0%|          | 0/150 [00:00<?, ?it/s]Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | bld_emb        | Embedding  | 800   
1 | bld_to_d       | Linear     | 1.2 K 
2 | bld_drop       | Dropout    | 0     
3 | source_project | Linear     | 3.1 K 
4 | target_project | Linear     | 3.1 K 
5 | dropout        | Dropout    | 0     
6 | encoder        | ModuleList | 5.5 M 
7 | decoder        | ModuleList | 5.5 M 
8 | output_layer   | Linear     | 387   
----------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
22.043    Total estimated model params size (MB)
  0%|          | 0/150 [02:08<?, ?it/s]

[I 2025-08-21 00:50:35,066] Trial 0 finished with value: 0.42607203125953674 and parameters: {'d_model_exp': 7, 'n_encoders': 1, 'top_k': 2, 'num_kernels': 4, 'd_ff_mult': 2, 'learning_rate': 0.0006729379254033839, 'dropout_rate': 0.1040268209025972, 'lr_decay': 0.9948039965565193, 'd_bld': 8, 'emb_dropout': 0.09949814801609339, 'gradient_clip_val': 1.4394564889044064, 'accumulate_grad_batches': 4, 'use_adaptive_beta': True}. Best is trial 0 with value: 0.42607203125953674.


Best trial: 0. Best value: 0.426072:   1%|          | 1/150 [02:08<5:20:19, 128.99s/it]Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | bld_emb        | Embedding  | 1.6 K 
1 | bld_to_d       | Linear     | 2.2 K 
2 | bld_drop       | Dropout    | 0     
3 | source_project | Linear     | 3.1 K 
4 | target_project | Linear     | 3.1 K 
5 | dropout        | Dropout    | 0     
6 | encoder        | ModuleList | 5.5 M 
7 | decoder        | ModuleList | 5.5 M 
8 | output_layer   | Linear     | 387   
----------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
22.047    Total estimated model params size (MB)
Best trial: 0. Best value: 0.426072

...

Best trial: 9. Best value: 0.201959:  14%|█▍        | 21/150 [18:47<1:26:37, 40.29s/it]Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | bld_emb        | Embedding  | 1.6 K 
1 | bld_to_d       | Linear     | 2.2 K 
2 | bld_drop       | Dropout    | 0     
3 | source_project | Linear     | 3.1 K 
4 | target_project | Linear     | 3.1 K 
5 | dropout        | Dropout    | 0     
6 | encoder        | ModuleList | 5.5 M 
7 | decoder        | ModuleList | 5.5 M 
8 | output_layer   | Linear     | 387   
----------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
22.047    Total estimated model params size (MB)
Best trial: 9. Best value: 0.201959

[I 2025-08-21 01:07:42,182] Trial 21 pruned. Trial was pruned at epoch 1.


Best trial: 9. Best value: 0.201959:  15%|█▍        | 22/150 [19:16<1:18:09, 36.63s/it]Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | bld_emb        | Embedding  | 800   
1 | bld_to_d       | Linear     | 1.2 K 
2 | bld_drop       | Dropout    | 0     
3 | source_project | Linear     | 3.1 K 
4 | target_project | Linear     | 3.1 K 
5 | dropout        | Dropout    | 0     
6 | encoder        | ModuleList | 5.5 M 
7 | decoder        | ModuleList | 5.5 M 
8 | output_layer   | Linear     | 387   
----------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
22.043    Total estimated model params size (MB)
Best trial: 9. Best value: 0.201959

[I 2025-08-21 01:07:45,127] Trial 22 finished with value: inf and parameters: {'d_model_exp': 7, 'n_encoders': 1, 'top_k': 2, 'num_kernels': 4, 'd_ff_mult': 2, 'learning_rate': 0.001553779841792309, 'dropout_rate': 0.09954350505567366, 'lr_decay': 0.9863993291691993, 'd_bld': 8, 'emb_dropout': 0.1174328663870561, 'gradient_clip_val': 1.1399339025834854, 'accumulate_grad_batches': 4, 'use_adaptive_beta': False}. Best is trial 9 with value: 0.201958566904068.


Best trial: 9. Best value: 0.201959:  15%|█▌        | 23/150 [19:19<56:08, 26.52s/it]  Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | bld_emb        | Embedding  | 800   
1 | bld_to_d       | Linear     | 1.2 K 
2 | bld_drop       | Dropout    | 0     
3 | source_project | Linear     | 3.1 K 
4 | target_project | Linear     | 3.1 K 
5 | dropout        | Dropout    | 0     
6 | encoder        | ModuleList | 5.5 M 
7 | decoder        | ModuleList | 5.5 M 
8 | output_layer   | Linear     | 387   
----------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
22.043    Total estimated model params size (MB)


In [ ]:
# 퍼블릭 SMAPE 평가에 필요한 전역 확인
assert 'val_pred_loader' in globals() and 'scaler' in globals() \
    and 'group_ids_val' in globals() and 'val_target_arrays' in globals(), \
    "Public SMAPE 평가에 필요한 전역 변수가 없습니다."

RUN_STAGE2 = True  # ← 튜닝을 “건너뛰려면” False 로만 바꾸면 됨

study_s2 = load_study(S2_PATH)
if (study_s2 is None) and RUN_STAGE2:
    study_s2 = optuna.create_study(
        sampler=optuna.samplers.TPESampler(seed=random_seed),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10),
        study_name="TimesNet_stage2_behavior",
        direction="minimize",  # 목적: Public SMAPE
    )
    study_s2.optimize(
        lambda tr: stage2_objective(tr, actual_source_dims, actual_target_dims, s1p),
        n_trials=80, show_progress_bar=True, gc_after_trial=True
    )
    with open(S2_PATH, "wb") as f:
        pickle.dump(study_s2, f)
elif study_s2 is None and not RUN_STAGE2:
    raise RuntimeError(f"RUN_STAGE2=False 이지만 저장된 Stage-2 스터디가 없습니다: {S2_PATH}")

best_s2 = study_s2.best_trial
print("S2 best public SMAPE:", best_s2.value)
print("S2 best params:", best_s2.params)
print("S2 attrs:", best_s2.user_attrs)


In [ ]:
# ============================== #
# Final training (TimesNet, Determinism OFF)
# ============================== #
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import CSVLogger
import numpy as np, torch, os, json, shutil, pickle
from datetime import datetime, timezone
from lightning.pytorch import Trainer

NONNEG_CLIP = True

# --- Stage-1/2 결과 사용 ---
p1 = best_s1.params
p2 = best_s2.params

# --- 구조 복원 (S1; TimesNet 전용) ---
# d_model
d_model = p1.get("d_model", 2 ** p1.get("d_model_exp", 7))  # fallback=128
# 깊이
n_encoders   = p1.get("n_encoders", 2)
n_decoders   = p1.get("n_decoders", max(1, n_encoders))
# TimesNet 고유
top_k        = p1.get("top_k", 3)
num_kernels  = p1.get("num_kernels", 6)
d_ff_tn      = p1.get("d_ff_tn", int(d_model * p1.get("d_ff_mult", 3)))

# 학습 관련 (S1)
learning_rate = p1.get("learning_rate", 1e-3)
dropout_rate  = p1.get("dropout_rate", 0.2)
lr_decay      = p1.get("lr_decay", 1.0)
grad_clip     = float(p1.get("gradient_clip_val", 1.0))
accum         = p1.get("accumulate_grad_batches", 1)
# 임베딩
d_bld        = p1.get("d_bld", 16)
emb_dropout  = p1.get("emb_dropout", 0.0)

# --- 동작 하이퍼 (S2) ---
beta_week         = p2.get("beta_week", 0.6)
alpha             = p2.get("alpha", 0.7)
use_adaptive_beta = p2.get("use_adaptive_beta", True)

# 실제 입력 차원
actual_source_dims = train_source_arrays[0].shape[1]
actual_target_dims = train_target_arrays[0].shape[1]

# --- 최종 하이퍼 (TimesNet) ---
final_hparams = {
    "source_length": SOURCE_LENGTH, "target_length": TARGET_LENGTH,
    "source_dims": actual_source_dims, "target_dims": actual_target_dims,
    "horizon_start": 0, "quantiles": [0.025, 0.5, 0.975],

    # 구조/용량
    "d_model": d_model,
    "n_encoders": n_encoders,
    "n_decoders": n_decoders,

    # 학습
    "learning_rate": learning_rate,
    "dropout_rate": dropout_rate,
    "lr_decay": lr_decay,

    # 임베딩
    "num_buildings": len(uniq_ids),
    "d_bld": d_bld,
    "emb_dropout": emb_dropout,

    # TimesNet 전용
    "top_k": top_k,
    "num_kernels": num_kernels,
    "d_ff_tn": d_ff_tn,

    # 동작 하이퍼
    "beta_week": beta_week,
    "alpha": alpha,
    "use_adaptive_beta": use_adaptive_beta,
}

# 데이터로더 (학습 전용 로더)
final_model = TimesNet(final_hparams)
final_train_data = SequenceDatasetWithBld(
    np.array(train_source_arrays), np.array(train_target_arrays), bld_ids=train_bld_ids_idx
)
final_train_loader = torch.utils.data.DataLoader(final_train_data, batch_size=batch_size, shuffle=True)

# 콜백/로거
os.makedirs(FINAL_DIR, exist_ok=True)
ckpt_cb = ModelCheckpoint(
    dirpath=FINAL_DIR, filename="final-{epoch:02d}-{val_loss:.4f}",
    monitor="val_loss", mode="min", save_top_k=1, save_last=True
)
es_cb  = EarlyStopping(monitor="val_loss", mode="min", patience=5)
lr_cb  = LearningRateMonitor(logging_interval="epoch")
csv_logger = CSVLogger(save_dir=FINAL_DIR, name="logs")

# 에폭은 S1 탐색 결과 재사용(없으면 30)
optimal_epochs = 50
print(f"✅ Using epochs: {optimal_epochs}")

# ***결정성 OFF
final_trainer = Trainer(
    max_epochs=optimal_epochs,
    accelerator="cpu", devices=1,
    logger=csv_logger, enable_progress_bar=True,
    precision=32,
    gradient_clip_val=grad_clip, accumulate_grad_batches=accum,
    callbacks=[ckpt_cb, es_cb, lr_cb],
    deterministic=False,
    benchmark=False,
)

assert 'val_loader' in globals(), "val_loader가 필요합니다."
final_trainer.fit(final_model, final_train_loader, val_loader)

best_ckpt_path = ckpt_cb.best_model_path
print(f"✅ Best checkpoint: {best_ckpt_path}")

# 로드 & 검증
loaded_final_model = TimesNet.load_from_checkpoint(
    best_ckpt_path, hyperparams_dict=final_hparams,
    map_location="cuda" if torch.cuda.is_available() else "cpu"
)
loaded_final_model.eval()
val_metrics = final_trainer.validate(loaded_final_model, dataloaders=val_loader, verbose=True)
print("Final val metrics:", val_metrics)

# 퍼블릭 창 재평가
with torch.inference_mode():
    raw = final_trainer.predict(loaded_final_model, dataloaders=val_pred_loader)
preds_raw_list, fut_ex_list = zip(*raw)
residual_q = torch.cat(preds_raw_list, dim=0).cpu().numpy().astype(np.float32)  # (N,H,Q)
fut_ex     = torch.cat(fut_ex_list,   dim=0).cpu().numpy().astype(np.float32)  # (N,H)

# 중앙 분위수 채널 인덱스
mid = getattr(loaded_final_model, "median_idx", 1)
yhat_scaled = residual_q[:, :, mid] + fut_ex
N = residual_q.shape[0]
group_ids_used = list(group_ids_val[:N])
y_scaled = np.stack([arr[:, -1] for arr in val_target_arrays], axis=0).astype(np.float32)[:N]
yhat_real = scaler.backtransform_preds(yhat_scaled, group_ids=group_ids_used)
y_real    = scaler.backtransform_preds(y_scaled,    group_ids=group_ids_used)

if NONNEG_CLIP:
    yhat_real = np.clip(yhat_real, 0.0, None)
    y_real    = np.clip(y_real,    0.0, None)

Hpub = min(int(PUBLIC_H), y_real.shape[1])
yhat_pub = yhat_real[:, -Hpub:]
y_pub    = y_real[:, -Hpub:]

smape_public = _smape_np(y_pub, yhat_pub, eps=EPS_PUBLIC)
bias_public  = float(np.nanmean(yhat_pub - y_pub))
print(f"🎯 Public-window SMAPE({Hpub}): {smape_public:.6f} | bias: {bias_public:.6f}")

# 메타 저장
final_ckpt_copy = os.path.join(FINAL_DIR, "final_best.ckpt")
if best_ckpt_path and os.path.exists(best_ckpt_path):
    shutil.copy2(best_ckpt_path, final_ckpt_copy)

meta = {
    "timestamp": datetime.now(timezone.utc).isoformat(),
    "final_hyperparams": final_hparams,
    "stage1_best": {"value": best_s1.value, "params": p1, "attrs": best_s1.user_attrs},
    "stage2_best": {"value": best_s2.value, "params": p2, "attrs": best_s2.user_attrs},
    "val_metrics": val_metrics,
    "public_window": {
        "H": Hpub, "smape": smape_public, "bias": bias_public,
        "eps": EPS_PUBLIC, "nonneg_clip": NONNEG_CLIP
    },
    "paths": {"best_ckpt": best_ckpt_path, "copied_ckpt": final_ckpt_copy, "save_dir": FINAL_DIR}
}
with open(os.path.join(FINAL_DIR, "final_meta.json"), "w") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print(f"🧾 메타 저장 완료: {os.path.join(FINAL_DIR, 'final_meta.json')}")


## Model testing

In [ ]:
# ===================================================================
# 1. Validation Set 예측 + 후처리 + 시각화/지표 (교체본)
# ===================================================================
import torch, numpy as np, pandas as pd, matplotlib.pyplot as plt, os, json

# ---- 토글/상수 ----
APPLY_PI_SHRINK = True
FINAL_PI_TAU    = 0.6
EPS_PUBLIC      = 1.0

print("Validation 데이터에 대한 성능 분석을 시작합니다...")

# 1) 예측
with torch.inference_mode():
    raw_outputs = final_trainer.predict(loaded_final_model, dataloaders=val_pred_loader)

preds_raw_list, fut_ex_list = zip(*raw_outputs)
residual_q = torch.cat(preds_raw_list, dim=0).cpu().numpy().astype(np.float32)  # (N,H,3) 가정
fut_ex     = torch.cat(fut_ex_list,   dim=0).cpu().numpy().astype(np.float32)   # (N,H)
assert residual_q.shape[:2] == fut_ex.shape[:2]
N, H, Q = residual_q.shape
assert Q == 3, f"예측 Q축이 3이 아닙니다: {Q}"

# 2) 잔차 -> 스케일된 실제값 (바이어스 보정 없음)
preds_raw_val = residual_q + fut_ex[:, :, None]   # (N,H,3) : [low, median, high] 가정

# 3) 역변환(원단위) & 물리 제약
#    - 기존 코드와 동일: val_target에서 건물번호 추출
if 'group_ids_val' not in globals() or group_ids_val is None or len(group_ids_val) != preds_raw_val.shape[0]:
    try:
        group_ids_val = [vt.iloc[0][BUILDING_COL] for vt in val_target]
    except Exception:
        raise RuntimeError("group_ids_val을 만들 수 없습니다. val_target 시퀀스에 BUILDING_COL이 있어야 합니다.")

preds_val_no = scaler.backtransform_preds(preds_raw_val, group_ids=group_ids_val)  # (N,H,3)
preds_val_no = np.clip(preds_val_no, 0.0, None)

# 4) (선택) PI 수축 - 원단위에서 적용해 비교 후, 좋아지면 반영
def _smape_np(y_true, y_pred, eps=1.0):
    y_true = np.asarray(y_true, np.float32)
    y_pred = np.asarray(y_pred, np.float32)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    if not np.any(m): return 1e9
    y_true = y_true[m]; y_pred = y_pred[m]
    return float(np.mean(2.0*np.abs(y_pred - y_true) /
                         (np.abs(y_true) + np.abs(y_pred) + eps)))

# 실제값(원단위) 배열 만들기: val_target 그대로 사용 (순서 가정 동일)
y_real_val = np.stack([df[TARGET_COL].to_numpy() for df in val_target], axis=0).astype(np.float32)
H_use = min(H, y_real_val.shape[1])
y_real_val = y_real_val[:, :H_use]
preds_val_no = preds_val_no[:, :H_use, :]

# (수축후 후보)
preds_val_sh = preds_val_no.copy()
if APPLY_PI_SHRINK:
    med = preds_val_sh[:, :, 1]
    preds_val_sh[:, :, 0] = med - FINAL_PI_TAU * (med - preds_val_sh[:, :, 0])
    preds_val_sh[:, :, 2] = med + FINAL_PI_TAU * (preds_val_sh[:, :, 2] - med)

# 성능 비교(중앙선)
sm_no = _smape_np(y_real_val, preds_val_no[:, :, 1], eps=EPS_PUBLIC)
sm_sh = _smape_np(y_real_val, preds_val_sh[:, :, 1], eps=EPS_PUBLIC)
use_shrink = APPLY_PI_SHRINK and (sm_sh <= sm_no)

preds_val = preds_val_sh if use_shrink else preds_val_no

# 정책 저장(선택)
if 'FINAL_DIR' not in globals():
    FINAL_DIR = "C:/Users/Daniel/DeepLearningEnergyForecasting/TimesNet_final_run"
os.makedirs(FINAL_DIR, exist_ok=True)
policy = {"pi_shrink": bool(use_shrink), "pi_tau": float(FINAL_PI_TAU if use_shrink else 0.0)}
with open(os.path.join(FINAL_DIR, "postproc_policy.json"), "w") as f:
    json.dump(policy, f, ensure_ascii=False, indent=2)
print("[POLICY] Saved:", policy, f"(SMAPE no={sm_no:.6f}, sh={sm_sh:.6f})")

# 5) DF 생성(네 기존 방식 유지)
val_source_with_ids = list(enumerate(val_source))
val_target_with_ids = list(enumerate(val_target))
df_preds_val   = predictions_to_dataframe(preds_val, val_target_with_ids)
df_actuals_val = test_sequences_to_dataframe(val_source_with_ids, val_target_with_ids)

# 6) 시각화
print("\n--- 전체 검증 기간 예측 결과 ---")
_ = plot_actual_predicted(df_actuals_val, df_preds_val, "TimesNet (Validation)")
plt.show()

print("\n--- 샘플 구간 예측 상세 분석 ---")
num_val_samples = len(val_target_with_ids)
indices_to_plot = [0, max(0, num_val_samples // 3 - 1), num_val_samples // 2, num_val_samples - 1] if num_val_samples >= 4 else list(range(num_val_samples))
for idx in indices_to_plot:
    _ = plot_sequence_preds(
        preds_val,
        val_source_with_ids, val_target_with_ids,
        "TimesNet (Validation)",
        target_col=TARGET_COL,
        sequence_index=idx
    )
    plt.show()

# 7) 지표
df_val_output_only = df_actuals_val[df_actuals_val["sequence"] == "output"].copy()
final_metrics = calculate_metrics(df_val_output_only, df_preds_val, model="TimesNet", target_col=TARGET_COL)
print("\n--- TimesNet Validation Set 성능 ---")
display(final_metrics)
print("------------------------------------")

# sanity
print(f"[CHK] mean(y_real_val): {float(np.nanmean(y_real_val)):.3f}")
print(f"[CHK] mean(pred_val_mid): {float(np.nanmean(preds_val[:, :, 1])):.3f}")
print(f"[CHK] mean(pred - true): {float(np.nanmean(preds_val[:, :, 1] - y_real_val)):.3f}")


In [ ]:
assert residual_q.ndim == 3 and fut_ex.ndim == 2, "shape mismatch"
assert residual_q.shape[:2] == fut_ex.shape[:2],  "N,H mismatch"

# --- Public 72h SMAPE 모니터링 (검증 셀 말미에 추가) ---
mid = loaded_final_model.median_idx  # ✅ 중앙 분위수 인덱스
y_scaled    = np.stack([arr[:, -1] for arr in val_target_arrays], axis=0).astype(np.float32)
yhat_scaled = residual_q[:, :, mid] + fut_ex   # ✅ 1 → mid
y_real    = scaler.backtransform_preds(y_scaled,    group_ids=group_ids_val)
yhat_real = scaler.backtransform_preds(yhat_scaled, group_ids=group_ids_val)
if NONNEG_CLIP:
    y_real = np.clip(y_real, 0, None); yhat_real = np.clip(yhat_real, 0, None)

H_pub = min(int(PUBLIC_H), y_real.shape[1])
smape_public = _smape_np(y_real[:, -H_pub:], yhat_real[:, -H_pub:], eps=EPS_PUBLIC)
bias_public  = float(np.nanmean(yhat_real[:, -H_pub:] - y_real[:, -H_pub:]))
print(f"\n--- Public {H_pub}h proxy ---  SMAPE: {smape_public:.6f} | Bias: {bias_public:.6f}")

In [ ]:
# Validation에서도(선택)
assert (preds_val >= 0.0).all(), "Validation 역변환 예측에 음수가 존재합니다."

# ===== 건물별 성능 요약: 병합 없이 확실히 맞추는 안전 버전 =====
def _smape(a, f, eps=1.0):
    a = np.asarray(a, dtype=np.float32)
    f = np.asarray(f, dtype=np.float32)
    return 2.0 * np.abs(f - a) / (np.abs(f) + np.abs(a) + eps)

rows = []
for i, (_, df_tgt) in enumerate(val_target_with_ids):
    # 각 시퀀스의 타임스탬프/건물/실제값
    # (create_transformer_sequences_for_prediction 에서 set_index('일시') 했던 구조 가정)
    times = df_tgt.index.to_numpy()
    bld   = df_tgt[BUILDING_COL].iloc[0]
    y_true = df_tgt[TARGET_COL].to_numpy()

    # 예측 (median)
    y_pred = preds_val[i, :, mid]

    # 혹시 길이가 다르면 안전하게 맞춰줌
    H = min(len(times), len(y_true), len(y_pred))
    if H == 0:
        continue

    rows.append(pd.DataFrame({
        "건물번호": bld,
        "time": times[:H],
        "actual": y_true[:H],
        "pred_point": y_pred[:H],
    }))

tmp = pd.concat(rows, ignore_index=True)

by_bld = tmp.groupby("건물번호").apply(
    lambda g: pd.Series({
        "SMAPE": float(_smape(g["actual"].values, g["pred_point"].values).mean()),
        "MAE": float(np.mean(np.abs(g["actual"].values - g["pred_point"].values))),
        "N": int(len(g))
    })
).sort_values("SMAPE")

by_bld["flag_smallN"] = (by_bld["N"] < 50)

print("\n--- 건물별 성능 요약 (상위/하위 10) ---")
display(by_bld.head(10))
display(by_bld.tail(10))

In [ ]:
# =========================================================
# ✅ (Validation용) baseline(fut_base) 안전 확보 + a/b 검사
# =========================================================
import numpy as np
import pandas as pd
import torch

mid = loaded_final_model.median_idx  # ✅
fut_base = preds_raw_val[:, :, mid] - residual_q[:, :, mid]
# (= fut_ex 와 동일해야 합니다. 필요하면 np.allclose(fut_base, fut_ex)로 확인 가능)

# ---------------------------------------------------------
# (a) 훈련 기준: 건물별 타깃 스케일 범위(min≈0, max≈1) 샘플링 확인
#   - train_target_scaled: [(id, arr), ...] 형태
#   - arr: (T, D_with_building), col0=건물번호, last=스케일된 타깃
# ---------------------------------------------------------
samples = []
for _, arr in train_target_scaled:
    bld = int(arr[0, 0])   # 0번 컬럼: 건물번호
    y   = arr[:, -1]       # 마지막 컬럼: 스케일된 타깃
    samples.append((bld, float(y.min()), float(y.max())))

chk_a = (
    pd.DataFrame(samples, columns=["건물번호", "y_min", "y_max"])
      .groupby("건물번호")
      .agg({"y_min": "min", "y_max": "max"})
      .reset_index()
)

print("훈련 기준 건물별 타깃 스케일 범위(최솟값/최댓값):")
display(chk_a.sample(min(10, len(chk_a))))   # 임의 10개 샘플
print("요약 통계:")
display(chk_a[["y_min", "y_max"]].describe())

# ---------------------------------------------------------
# (b) Validation에서 '진짜 잔차' 표준편차(학습이 죽지 않는지) 확인
#   - val_target_arrays: 모델 입력(건물번호 제거), last=스케일된 타깃
#   - fut_base: weekly-blend baseline (스케일 공간)
# ---------------------------------------------------------
residuals = []
for i in range(len(val_target_arrays)):
    y_true_scaled = val_target_arrays[i][:, -1]  # (H,)
    base_scaled   = fut_base[i]                  # (H,)
    H = min(len(y_true_scaled), len(base_scaled))
    if H > 0:
        residuals.append(y_true_scaled[:H] - base_scaled[:H])

r = np.concatenate(residuals) if residuals else np.array([], dtype=np.float32)
if r.size == 0:
    print("[Validation] 잔차를 계산할 데이터가 없습니다.")
else:
    print(f"[Validation] true residual std: {r.std():.6f}  (mean={r.mean():.6f})")
    print("분위수:", np.percentile(r, [0, 1, 5, 25, 50, 75, 95, 99, 100]))


In [ ]:
# ===================================================================
# 2) Test Set 최종 예측 + 제출파일 생성 (일관성/파일 안전 적용본)
# ===================================================================
import torch, numpy as np, pandas as pd, os, json, time
from datetime import datetime

print("--- [3/3] 최종 제출 파일 생성 시작 ---")

# ---------- 0) RUN 식별자/출력 디렉토리 ----------
# FINAL_DIR 은 학습/검증 셀에서 만든 디렉토리.
# 여기에 run_id 하위 폴더를 만들어 모든 산출물을 분리 저장
if 'FINAL_DIR' not in globals() or not FINAL_DIR:
    FINAL_DIR = "C:/Users/Daniel/DeepLearningEnergyForecasting/TimesNet_final_run"
os.makedirs(FINAL_DIR, exist_ok=True)

# 가능하면 meta에서 동일 run을 재사용, 없으면 새 RUN_ID
meta_path = os.path.join(FINAL_DIR, "final_meta.json")
if os.path.exists(meta_path):
    try:
        with open(meta_path, "r") as f:
            _meta = json.load(f)
        RUN_ID = _meta.get("timestamp", datetime.utcnow().isoformat()).replace(":", "").replace("-", "")[:15]
        print(f"[INFO] Using RUN_ID from meta: {RUN_ID}")
    except Exception:
        RUN_ID = datetime.now().strftime("%Y%m%d-%H%M%S")
        print(f"[WARN] meta 읽기 실패 → new RUN_ID: {RUN_ID}")
else:
    RUN_ID = datetime.now().strftime("%Y%m%d-%H%M%S")
    print(f"[INFO] meta 없음 → new RUN_ID: {RUN_ID}")

RUN_DIR = os.path.join(FINAL_DIR, f"run_{RUN_ID}")
os.makedirs(RUN_DIR, exist_ok=True)
SUBMIT_DIR = os.path.join(RUN_DIR, "submit")
os.makedirs(SUBMIT_DIR, exist_ok=True)

# ---------- 1) 스케일 변환 (SequenceScaler 유지) ----------
infer_source_with_ids = list(enumerate(infer_src))
infer_target_with_ids = list(enumerate(infer_tgt))

source_scaled_final = scaler.transform(infer_source_with_ids)
target_scaled_final = scaler.transform(infer_target_with_ids)

# 배열화 + (타깃 제외: covariates-only) — 건물번호(첫 열) 제거
source_arrays_final = [arr[:, 1:]   for _, arr in source_scaled_final]
target_arrays_final = [arr[:, 1:-1] for _, arr in target_scaled_final]

print("Test source D:", source_arrays_final[0].shape[1])
print("Test target-cov D:", target_arrays_final[0].shape[1])

# ---------- 2) bld_ids (학습 때 만든 infer_bld_ids_idx 사용) ----------
final_test_data = SequenceDatasetWithBld(
    np.array(source_arrays_final),
    np.array(target_arrays_final),
    bld_ids=infer_bld_ids_idx
)
final_test_loader = torch.utils.data.DataLoader(final_test_data, batch_size=batch_size, shuffle=False)

# ---------- 3) 예측 실행 ----------
loaded_final_model.eval()
with torch.inference_mode():
    raw_outputs_test = final_trainer.predict(loaded_final_model, dataloaders=final_test_loader)

preds_raw_list_test, fut_ex_list_test = zip(*raw_outputs_test)
residual_q_test = torch.cat(preds_raw_list_test, dim=0).cpu().numpy().astype(np.float32)   # (N,H,3)
fut_ex_test     = torch.cat(fut_ex_list_test,   dim=0).cpu().numpy().astype(np.float32)    # (N,H)
assert residual_q_test.shape[:2] == fut_ex_test.shape[:2], "N,H mismatch"
N, H, Q = residual_q_test.shape
assert Q == 3, f"예측 Q축이 3이 아닙니다: {Q}"

# ---------- 4) 잔차 -> 스케일된 실제값 ----------
preds_raw_test = residual_q_test + fut_ex_test[:, :, None]   # (N,H,3)

# ---------- 5) 역변환(원단위) & 물리 제약 ----------
group_ids_test = [vt.iloc[0][BUILDING_COL] for vt in infer_tgt]
if len(group_ids_test) != preds_raw_test.shape[0]:
    print(f"[경고] group_ids_test 길이 불일치: {len(group_ids_test)} vs {preds_raw_test.shape[0]}")
preds_test = scaler.backtransform_preds(preds_raw_test, group_ids=group_ids_test)  # (N,H,3)
preds_test = np.clip(preds_test, 0.0, None)

# ---------- 6) Validation에서 저장한 postproc policy 적용 (원단위에서만 PI 수축) ----------
policy_path = os.path.join(FINAL_DIR, "postproc_policy.json")
policy_used = {"found": False}
if os.path.exists(policy_path):
    with open(policy_path, "r") as f:
        pol = json.load(f)
    policy_used = {"found": True, **pol}
    _pi_shrink = bool(pol.get("pi_shrink", False))
    _pi_tau    = float(pol.get("pi_tau", 0.0))
    print("[POLICY] Loaded:", pol)
    if _pi_shrink and _pi_tau > 0.0:
        med = preds_test[:, :, 1]
        preds_test[:, :, 0] = med - _pi_tau * (med - preds_test[:, :, 0])
        preds_test[:, :, 2] = med + _pi_tau * (preds_test[:, :, 2] - med)
else:
    print("[POLICY] Not found, proceed without PI shrink change.")

# ---------- 7) 제출 파일 생성 (median 사용) ----------
sub = pd.read_csv(SAMPLE_SUB_PATH)
expected_len = N * H
if len(sub) != expected_len:
    raise ValueError(f"제출 길이 불일치: {len(sub)} vs 예상 {expected_len}")

out_col = "answer" if "answer" in sub.columns else ("prediction" if "prediction" in sub.columns else "answer")
sub[out_col] = preds_test[:, :, 1].reshape(-1).astype(np.float32)

# 파일명에 RUN_ID 부여 + RUN 디렉토리에 보관(재현성)
submission_filename = f"submission_TimesNet_{RUN_ID}.csv"
submission_path = os.path.join(SUBMIT_DIR, submission_filename)
sub.to_csv(submission_path, index=False)

# 편의상 작업 디렉토리에도 동일 파일 복사
cwd_submission_path = os.path.join(os.getcwd(), submission_filename)
sub.to_csv(cwd_submission_path, index=False)

print(f"✅ 제출 파일 생성 완료: {submission_path}")
print(f"✅ 작업 디렉토리 복사본: {cwd_submission_path}")
print(f"[SUBMIT] shape: N={N}, H={H}, Q={Q}, out_col={out_col}, mean(median)={float(np.nanmean(preds_test[:,:,1])):.3f}")

# ---------- 8) 제출 산출물 provenance 저장 ----------
provenance = {
    "run_id": RUN_ID,
    "final_dir": FINAL_DIR,
    "run_dir": RUN_DIR,
    "submission_path": submission_path,
    "submission_filename": submission_filename,
    "timestamp": time.ctime(),
    "policy_used": policy_used,
    "N": int(N), "H": int(H), "Q": int(Q),
    "median_mean": float(np.nanmean(preds_test[:,:,1])),
}
with open(os.path.join(SUBMIT_DIR, "provenance.json"), "w") as f:
    json.dump(provenance, f, ensure_ascii=False, indent=2)
print("[PROVENANCE] Saved:", os.path.join(SUBMIT_DIR, "provenance.json"))

# 미리보기
display(sub.head())


In [ ]:
# 7) 4개 건물만 확대 시각화 (간단 교체본)
print("\n--- 4개 건물 예측 상세 분석 ---")

# (선택) 특정 4개 건물번호 지정 — 지정 안 하면 검증 세트에서 앞의 4개 서로 다른 건물 자동 선택
MANUAL_BUILDINGS = []  # 예: [101, 205, 309, 412]

# 건물번호 → 시퀀스 인덱스 매핑
bld_to_idx = { df[BUILDING_COL].iloc[0]: i for i, (_, df) in enumerate(val_target_with_ids) }

if MANUAL_BUILDINGS:
    indices_to_plot = [bld_to_idx[b] for b in MANUAL_BUILDINGS if b in bld_to_idx][:4]
else:
    # 검증 세트에서 서로 다른 건물 4개 자동 선택
    seen = set()
    indices_to_plot = []
    for i, (_, df) in enumerate(val_target_with_ids):
        b = df[BUILDING_COL].iloc[0]
        if b not in seen:
            seen.add(b)
            indices_to_plot.append(i)
            if len(indices_to_plot) == 4:
                break

for idx in indices_to_plot:
    _ = plot_sequence_preds(
        preds_val,
        val_source_with_ids, val_target_with_ids,
        "TimesNet (Validation)",
        target_col=TARGET_COL,
        sequence_index=idx
    )
    plt.show()

Keep in mind that for the median quantile, pinball loss equals MAE / 2.